In [1]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

df_train = pd.read_csv("/kaggle/input/titanic/train.csv")
df_train.head()

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
df_train.info()
df_train.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [3]:
# La columna 'Cabin' tiene demasiados valores vacion por lo que la eliminamos
df_train.drop('Cabin', axis=1, inplace=True)

In [4]:
# La columna 'Embarked' solo tiene 2 valores vacíos, vamos a rellenarlos con la moda
print(df_train['Embarked'].mode())
df_train['Embarked'] = df_train['Embarked'].fillna('S')

0    S
Name: Embarked, dtype: object


In [5]:
# La columna 'Age' tiene demasiados valores vacíos pero si la borramos perderíamos
# también mucha información, lo óptimo es rellenarlos con la media de edad
media_edad = df_train['Age'].mean()
print(f"La edad media es: {media_edad}")

df_train['Age'] = df_train['Age'].fillna(media_edad)

La edad media es: 29.69911764705882


In [6]:
# Comprobamos de nuevo la suma de nulos
df_train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [7]:
# Tenemos que traducir los valores de texto a numéricos 
# Para la columna 'Sex'-> female=0 y male=1
df_train['Sex'] = df_train['Sex'].map({'female': 0, 'male': 1})

# Para la columna 'Embarked' -> S=0, C=1 y Q=2
df_train['Embarked'] = df_train['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

In [8]:
# Eliminamos las columnas que no nos aportan ningún tipo de info útil
columnas_inutiles = ['PassengerId', 'Name', 'Ticket']
df_train.drop(columnas_inutiles, axis=1, inplace=True)

df_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,0
1,1,1,0,38.0,1,0,71.2833,1
2,1,3,0,26.0,0,0,7.9250,0
3,1,1,0,35.0,1,0,53.1000,0
4,0,3,1,35.0,0,0,8.0500,0


In [9]:
# Separamos la tabla en dos, 'y' contendrá la variable que queremos producir
# y 'x' contrendrá el resto de las columnas

y = df_train['Survived']
x = df_train.drop('Survived', axis=1)

In [10]:
from sklearn.tree import DecisionTreeClassifier

modelo = DecisionTreeClassifier()
modelo.fit(x, y)

DecisionTreeClassifier()

In [11]:
puntuacion = modelo.score(x, y)
print(f"La precisión del modelo en los datos de entrenamiento es: {puntuacion * 100:.2f}%")

La precisión del modelo en los datos de entrenamiento es: 98.20%


El modelo en este punto nos está dando una puntuación de 98.20%. Este no es un dato realista. Esto es un ejemplo de overfitting.
El modelo no ha aprendido predecir, digamos que ha "memorizado" los datos que teníamos.

Por ello a continuación, vamos a usar el datos de pasajeros que no conoce.
Leemos el dataset, aplicamos la misma limpieza realizada anteriormente y usamos nuestro modelo.

In [12]:
df_test = pd.read_csv("/kaggle/input/titanic/test.csv")

# Rellenamos o eliminamos los valores vacíos
df_test.drop('Cabin', axis=1, inplace=True)
df_test['Age'] = df_test['Age'].fillna(media_edad)
df_test['Embarked'] = df_test['Embarked'].fillna('S')

media_fare = df_test['Fare'].mean()
df_test['Fare'] = df_test['Fare'].fillna(media_fare)

# Columnas con valores de texto las pasamos a numéricos
df_test['Sex'] = df_test['Sex'].map({'female': 0, 'male': 1})
df_test['Embarked'] = df_test['Embarked'].map({'S':0, 'C': 1, 'Q': 2})

# Eliminamos las columnas que no nos importan info
# Guardamos aparte el ID para el envío final
passenger_ID = df_test['PassengerId']
columnas_inutiles = ['PassengerId','Name', 'Ticket']
df_test.drop(columnas_inutiles, axis=1, inplace=True)

df_test.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [13]:
predicciones = modelo.predict(df_test)
print(predicciones[0:10])

[0 0 1 1 1 0 0 0 1 0]


In [14]:
# Creamos un nuevo DataFrame para la entrega en Kaggle
entrega = pd.DataFrame({
    "PassengerId": passenger_ID,
    "Survived": predicciones
})

entrega.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,1
4,896,1


In [15]:
entrega.to_csv("entrega.csv", index=False)